In [177]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/ddx/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [178]:
import numpy as np
import pandas as pd

import json
import glob


import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import Phrases
from gensim.models.phrases import Phraser

import spacy
from nltk.corpus import stopwords

import pyLDAvis
import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)



In [179]:
def load_data(file):
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

def write_data(file, data):
    with open(file, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4)

In [180]:
stopwords = stopwords.words("english")
# stopwords

In [181]:
data = pd.read_csv('data.csv')['Description']

data.head()

0    Situated in a picturesque valley at the foothi...
1     The city of Gangneung is developing an inclus...
2    Casablanca is a major centre for media arts an...
3     Valencia emphasizes the essential role of des...
4     Veliky Novgorod Creative City has made signif...
Name: Description, dtype: object

In [182]:
data[5][0:90]

'\xa0Ouarzazate is considered to have a special place in the minds of film lovers. As one of t'

In [183]:
text = "\xa0Da Lat's economic growth is driven by its creative sector."
length = len(text)
print(length)

59


In [184]:
# !python3 -m spacy download en_core_web_sm
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:90])

situate picturesque valley foothill meeting point ancient civilization unique location inf


In [185]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['situate', 'picturesque', 'valley', 'foothill', 'meeting', 'point', 'ancient', 'civilization', 'unique', 'location', 'influence', 'city', 'design', 'offer', 'harmonious', 'blend', 'modern', 'technology', 'architectural', 'tradition']


In [186]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 4)]
active


In [187]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [188]:
pyLDAvis.enable_notebook()

vis = gensimvis.prepare(lda_model, corpus, id2word, mds="mmds", R=10, n_jobs=1)
vis


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.277615 -0.014987       1        1  35.935984
5     -0.139653 -0.233512       2        1  25.906209
7     -0.240254 -0.123897       3        1  20.728892
4     -0.199638  0.159283       4        1  15.563668
3      0.258958 -0.150493       5        1   1.751513
8      0.123154  0.072830       6        1   0.041037
0      0.119527  0.075669       7        1   0.032339
2      0.119218  0.070628       8        1   0.025375
9      0.118149  0.072248       9        1   0.007586
6      0.118153  0.072230      10        1   0.007396, topic_info=             Term        Freq       Total Category  logprob  loglift
331         music  638.000000  638.000000  Default  10.0000  10.0000
383          film  213.000000  213.000000  Default   9.0000   9.0000
21         design  664.000000  664.000000  Default   8.0000   8.0000
394    production  229.000000  229.000000  Default   7.0000   7.0000
45       industry  507.000000  507.000000  Default   6.0000   6.0000
...           ...         ...         ...      ...      ...      ...
2043  businessman    0.000981    1.011414  Topic10  -8.4235   2.5742
119      cultural    0.000981  781.302087  Topic10  -8.4235  -4.0754
2058        thick    0.000981    1.011415  Topic10  -8.4235   2.5742
2059   topography    0.000981    1.011415  Topic10  -8.4235   2.5742
136          food    0.000981  219.370048  Topic10  -8.4235  -2.8052

[181 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
921       5  1.002400      adopt
1603      3  1.005515  animation
190       1  0.636493        art
190       2  0.031363        art
190       3  0.321014        art
...     ...       ...        ...
358       3  0.176326      world
358       4  0.078367      world
358       5  0.034286      world
646       2  0.999851     writer
920       2  0.984360    writing

[197 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 6, 8, 5, 4, 9, 1, 3, 10, 7])

In [189]:
# BIGRAMS AND TRIGRAMS
bigrams_phrases = Phrases(data_words, min_count=5, threshold=50)
trigram_phrases = Phrases(bigrams_phrases[data_words], threshold=50)

# Creating phrasers for efficiency
bigram = Phraser(bigrams_phrases)
trigram = Phraser(trigram_phrases)

# Functions to create bigrams and trigrams
def make_bigrams(texts):
    return [bigram[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram[bigram[doc]] for doc in texts]

# Applying the bigram and trigram models
data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

# Printing the first document's bigrams/trigrams
print(data_bigrams_trigrams[0])


['situate', 'picturesque', 'valley', 'foothill', 'meeting_point', 'ancient', 'civilization', 'unique', 'location', 'influence', 'city', 'design', 'offer', 'harmonious', 'blend', 'modern', 'technology', 'architectural', 'tradition', 'city', 'centre', 'design', 'renowned', 'innovative', 'project', 'field', 'design', 'public_private', 'sector', 'professional', 'designer', 'play', 'active', 'role', 'today', 'design', 'industry', 'encompass', 'enterprise', 'employ', 'skilled', 'individual', 'showcase', 'commitment', 'urban', 'design', 'host', 'exhibition', 'gather', 'lead', 'local', 'national', 'international', 'designer', 'follower', 'project', 'implement', 'strengthen', 'tie', 'graduate', 'student', 'host', 'various', 'competition', 'design', 'combine', 'traditional', 'contemporary', 'trend', 'project', 'form', 'creative', 'union', 'professional', 'young', 'aspire', 'designer', 'connect', 'municipality', 'community', 'field', 'design', 'dedicate', 'provide', 'training', 'opportunity', 'as

In [190]:
print(data_bigrams_trigrams[0:50])

[['situate', 'picturesque', 'valley', 'foothill', 'meeting_point', 'ancient', 'civilization', 'unique', 'location', 'influence', 'city', 'design', 'offer', 'harmonious', 'blend', 'modern', 'technology', 'architectural', 'tradition', 'city', 'centre', 'design', 'renowned', 'innovative', 'project', 'field', 'design', 'public_private', 'sector', 'professional', 'designer', 'play', 'active', 'role', 'today', 'design', 'industry', 'encompass', 'enterprise', 'employ', 'skilled', 'individual', 'showcase', 'commitment', 'urban', 'design', 'host', 'exhibition', 'gather', 'lead', 'local', 'national', 'international', 'designer', 'follower', 'project', 'implement', 'strengthen', 'tie', 'graduate', 'student', 'host', 'various', 'competition', 'design', 'combine', 'traditional', 'contemporary', 'trend', 'project', 'form', 'creative', 'union', 'professional', 'young', 'aspire', 'designer', 'connect', 'municipality', 'community', 'field', 'design', 'dedicate', 'provide', 'training', 'opportunity', 'a

In [191]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
# print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

In [192]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [193]:
pyLDAvis.enable_notebook()

vis = gensimvis.prepare(lda_model, corpus, id2word, mds="mmds", R=15, n_jobs=1)
vis


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.131807  0.234849       1        1  68.357990
4     -0.298503  0.047527       2        1  14.104721
3      0.108754  0.251512       3        1   7.373594
1      0.313775  0.040114       4        1   5.032161
6     -0.144684 -0.218163       5        1   2.756795
0      0.121910 -0.237806       6        1   2.017088
2      0.024050 -0.059173       7        1   0.246172
7      0.006505 -0.058861       8        1   0.111480, topic_info=            Term        Freq       Total Category  logprob  loglift
325        music  582.000000  582.000000  Default  15.0000  15.0000
377         film  199.000000  199.000000  Default  14.0000  14.0000
637     literary  176.000000  176.000000  Default  13.0000  13.0000
879        craft  232.000000  232.000000  Default  12.0000  12.0000
447   literature  137.000000  137.000000  Default  11.0000  11.0000
...          ...         ...         ...      ...      ...      ...
2677   prevalent    0.070086    0.706357   Topic8  -6.6135   4.4887
1601   furniture    0.206587    4.916026   Topic8  -5.5326   3.6295
2024       bread    0.119426    2.006477   Topic8  -6.0806   3.9776
2454   abundance    0.071459    0.827192   Topic8  -6.5941   4.3502
1803      search    0.070086    1.005520   Topic8  -6.6135   4.1355

[214 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1747      1  0.886982  advantage
1747      7  0.110873  advantage
2022      4  0.958223    amateur
1643      5  0.928182  animation
292       1  0.988931     annual
...     ...       ...        ...
353       2  0.072434       year
353       3  0.172032       year
353       4  0.162978       year
98        1  0.988227      young
98        3  0.010294      young

[273 rows x 3 columns], R=15, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 5, 4, 2, 7, 1, 3, 8])

In [197]:
test_doc = corpus[-1]
vector = lda_model[test_doc]
print(vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)
new_vector = Sort(vector)
print (new_vector)

[(1, 0.38897592), (3, 0.02589855), (4, 0.013317852), (5, 0.5501712), (6, 0.020905493)]
[(5, 0.5501712), (1, 0.38897592), (3, 0.02589855), (6, 0.020905493), (4, 0.013317852)]


In [195]:
lda_model.save('./Models/test_model.model')

In [198]:
new_model = gensim.models.ldamodel.LdaModel.load('./Models/test_model.model')

test_doc = corpus[-1]
vector = new_model[test_doc]
print(vector)
def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)
new_vector = Sort(vector)
print (new_vector)

[(1, 0.38897592), (3, 0.02589855), (4, 0.013317852), (5, 0.5501712), (6, 0.020905493)]
[(5, 0.5501712), (1, 0.38897592), (3, 0.02589855), (6, 0.020905493), (4, 0.013317852)]
